# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,codrington,-38.2667,141.9667,10.01,99,78,1.63,AU,2023-11-07 19:00:35
1,1,isoka,-10.1606,32.6335,21.46,69,100,0.33,ZM,2023-11-07 19:00:38
2,2,puerto natales,-51.7236,-72.4875,10.25,53,75,10.29,CL,2023-11-07 19:00:40
3,3,yellowknife,62.4560,-114.3525,-4.77,97,100,3.09,CA,2023-11-07 19:00:43
4,4,klyuchi,52.2667,79.1667,-1.89,77,69,2.14,RU,2023-11-07 19:00:45


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City", alpha=0.6,
    hover_cols = ["City", "Humidity"]
)

# Display the map plot
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
clean_city_data = city_data_df.dropna()
# Set criteria
vaca_max_humid = city_data_df['Humidity']<45
vaca_min_humid = city_data_df['Humidity']>20
vaca_clouds = city_data_df['Cloudiness']<10
vaca_temp_min = city_data_df['Max Temp']>23
vaca_temp_max = city_data_df['Max Temp']<30
vaca_wind_max = city_data_df['Wind Speed']<5
vaca_wind_min = city_data_df['Wind Speed']>1
# Narrow down cities that fit criteria
vaca_cities = city_data_df[vaca_clouds & vaca_temp_max & vaca_temp_min & vaca_max_humid & vaca_min_humid & vaca_wind_max & vaca_wind_min]

# Display sample data
vaca_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
65,65,dahlonega,34.5326,-83.9849,26.06,43,0,3.44,US,2023-11-07 19:00:00
87,87,eufaula,31.8913,-85.1455,27.30,40,0,2.06,US,2023-11-07 19:02:49
90,90,greenville,35.6127,-77.3663,27.09,40,0,3.60,US,2023-11-07 19:02:53
107,107,fort bragg,35.1390,-79.0060,27.58,34,0,4.12,US,2023-11-07 19:03:21
240,240,al jawf,29.5000,38.7500,23.25,37,0,2.32,SA,2023-11-07 19:06:29
266,266,jesup,31.6074,-81.8854,27.96,28,0,2.06,US,2023-11-07 19:07:01
346,346,pokaran,26.9167,71.9167,27.02,25,1,3.26,IN,2023-11-07 19:08:41
375,375,siwa oasis,29.2041,25.5195,25.01,41,0,2.33,EG,2023-11-07 19:09:17
383,383,dawesville,30.9224,-84.0091,27.42,33,0,1.54,US,2023-11-07 19:09:28
479,479,letlhakeng,-24.0944,25.0298,23.58,26,0,2.67,BW,2023-11-07 19:11:54


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame({
    "City" : vaca_cities["City"], 
    'Country' : vaca_cities['Country'], 
    "Lat" : vaca_cities['Lat'], 
    "Lng" : vaca_cities["Lng"], 
    "Humidity" : vaca_cities["Humidity"]
                })

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
65,dahlonega,US,34.5326,-83.9849,43,
87,eufaula,US,31.8913,-85.1455,40,
90,greenville,US,35.6127,-77.3663,40,
107,fort bragg,US,35.1390,-79.0060,34,
240,al jawf,SA,29.5000,38.7500,37,
266,jesup,US,31.6074,-81.8854,28,
346,pokaran,IN,26.9167,71.9167,25,
375,siwa oasis,EG,29.2041,25.5195,41,
383,dawesville,US,30.9224,-84.0091,33,
479,letlhakeng,BW,-24.0944,25.0298,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000
limit = 5
params = {
    "categories" : 'accommodation.hotel',
    "limit": limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
  
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
dahlonega - nearest hotel: Days Inn
eufaula - nearest hotel: Days Inn by Wyndham Eufaula AL
greenville - nearest hotel: Quality Inn
fort bragg - nearest hotel: Airborne Inn Lodging
al jawf - nearest hotel: No hotel found
jesup - nearest hotel: Days Inn
pokaran - nearest hotel: Pokaran Desert Resort
siwa oasis - nearest hotel: فندق الكيلانى
dawesville - nearest hotel: Baymont by Wyndham Thomasville
letlhakeng - nearest hotel: No hotel found
pilani - nearest hotel: VFAST


,City,Country,Lat,Lng,Humidity,Hotel Name
65,dahlonega,US,34.5326,-83.9849,43,Days Inn
87,eufaula,US,31.8913,-85.1455,40,Days Inn by Wyndham Eufaula AL
90,greenville,US,35.6127,-77.3663,40,Quality Inn
107,fort bragg,US,35.1390,-79.0060,34,Airborne Inn Lodging
240,al jawf,SA,29.5000,38.7500,37,No hotel found
266,jesup,US,31.6074,-81.8854,28,Days Inn
346,pokaran,IN,26.9167,71.9167,25,Pokaran Desert Resort
375,siwa oasis,EG,29.2041,25.5195,41,فندق الكيلانى
383,dawesville,US,30.9224,-84.0091,33,Baymont by Wyndham Thomasville
479,letlhakeng,BW,-24.0944,25.0298,26,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City", alpha=0.6,
    hover_cols = ["City", "Humidity", "Country", "Hotel Name"]
)
# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)